## Beispiel-Thread

In [15]:
import praw
import json
import collections
from datetime import datetime

secret = json.load(open('.secret.json'))
reddit = praw.Reddit(client_id=secret["client_id"], 
                     client_secret= secret["client_secret"], 
                     user_agent='dataviz:v0.1 (by /u/nobody22)')

In [4]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Testing Reddit bots with Betamax
Please explain this if statement to me
Thankyou r/Learnpython
Working on subset of pandas dataframe
I scraped the data, now what?
How to scrap web element ID that kept changing upon page reload
Chatbot with Neural Network (Tensorflow). Where to start?
Adding a webportal to my program
Beginner exploring IDEs and text editors; advice needed!
Python 3 SMTP Attachment Error


<blockquote class="reddit-card" data-card-created="1493683113"><a href="https://www.reddit.com/r/news/comments/3v6iq7/authorities_respond_to_20_victim_shooting/?ref=share&ref_source=embed">Authorities Respond to “20 Victim Shooting” Incident in San Bernardino: Fire Dept.</a> from <a href="http://www.reddit.com/r/news">news</a></blockquote>
<script async src="//embed.redditmedia.com/widgets/platform.js" charset="UTF-8"></script>

## TODO: get all comments

In [19]:
submission.comments.replace_more
top = submission.comments[0]
top

Comment(id='cxkqnwq')

In [13]:
def get_comment_data(comment):
    c = {}
    c['id'] = comment.id
    c['created_utc'] = comment.created_utc
    if comment.author:
        c['author_name'] = comment.author.name 
#     if comment.body:
#         c['body'] = comment.body  
    if comment.score:
        c['score'] = comment.score
    comment.replies.replace_more(limit=0)
    if comment.replies:
        c['children'] = [get_comment_data(r) for r in comment.replies]
    return c

In [14]:
submission = reddit.submission(url='https://www.reddit.com/r/news/comments/3v6iq7/authorities_respond_to_20_victim_shooting/')

In [20]:
str(datetime.fromtimestamp(submission.created_utc))
submission.fullname

't3_3v6iq7'

In [21]:
print(top.link_id)
print(top.replies[0].link_id)

t3_3v6iq7
t3_3v6iq7


In [22]:
not_replaced = submission.comments.replace_more(limit=0)

In [23]:
comment_data = [get_comment_data(root_comment) for root_comment in submission.comments]

In [24]:
more = not_replaced[0]
more.comments()

[Comment(id='cxl57n3'),
 <MoreComments count=9, children=['cxl5bxo', 'cxl5atu', 'cxl58ej', '...']>,
 Comment(id='cxlgax8'),
 <MoreComments count=19, children=['cxlghdk', 'cxlgdr7']>,
 Comment(id='cxlbpun'),
 <MoreComments count=17, children=['cxlbxsz', 'cxlbvae', 'cxlbstp', '...']>,
 Comment(id='cxks4zf'),
 <MoreComments count=1, children=['cxks6xw']>,
 Comment(id='cxkwqka'),
 <MoreComments count=2, children=['cxkwzyb']>,
 Comment(id='cxkzw4j'),
 <MoreComments count=6, children=['cxkzyw1', 'cxl028b', 'cxkzygb', '...']>,
 Comment(id='cxl27yw'),
 Comment(id='cxl295y'),
 <MoreComments count=1, children=['cxl2bhv']>,
 Comment(id='cxl2a3d'),
 <MoreComments count=1, children=['cxl290d']>,
 Comment(id='cxl32f3'),
 <MoreComments count=2, children=['cxl5zj4', 'cxl3jod']>,
 Comment(id='cxl47n4'),
 <MoreComments count=14, children=['cxl4ctd', 'cxl7h9i', 'cxl4a0h', '...']>,
 Comment(id='cxlzcg9'),
 <MoreComments count=8, children=['cxm7rtm', 'cxlzv9x']>,
 Comment(id='cxkuonf'),
 <MoreComments coun

In [25]:
submission_data = {
                    'id': submission.fullname,
                    'title': submission.title,
                    'created_utc': submission.created_utc,
                    'author_name' : submission.author.name,
                    'children': [],
                    'parent_id': ""
                  }
submission_data['children'] = comment_data
json.dump(submission_data, open('test_submission.json', mode='w'), indent=True)

In [27]:
def rec_sum(c):
    if not "children" in c:
        return 1
    else:
        return 1 + sum([rec_sum(child) for child in c["children"]])
rec_sum(submission_data)

496

In [28]:
len(submission_data["children"])

228

## BigQueryJSON to JSON

In [29]:
from path import Path
def fullname(jsonobj):
    jsonobj["id"] = "t1_" + jsonobj["id"]
    return jsonobj
for file in ["data/results-20170502-230026.json"]:
    with open(file) as bfile:
        data = [fullname(json.loads(line)) for line in bfile.readlines()]
    with open("data/data.json", mode='w') as jfile:
        data.append(submission_data)
        json.dump(data, jfile)

In [30]:
parents = set([c['parent_id'] for c in data])


In [31]:
len([c for c in data if c['parent_id'] in parents])

2001

In [33]:
data[:5]

[{'created_utc': '1449084244',
  'id': 't1_cxkpih0',
  'parent_id': 't3_3v6iq7',
  'score': '6'},
 {'created_utc': '1449084454',
  'id': 't1_cxkpo3w',
  'parent_id': 't3_3v6iq7',
  'score': '-34'},
 {'created_utc': '1449084459',
  'id': 't1_cxkpo9n',
  'parent_id': 't3_3v6iq7',
  'score': '3'},
 {'created_utc': '1449084528',
  'id': 't1_cxkpq2r',
  'parent_id': 't3_3v6iq7',
  'score': '-2'},
 {'created_utc': '1449084608',
  'id': 't1_cxkps5t',
  'parent_id': 't1_cxkpo3w',
  'score': '-6'}]

In [34]:
import pandas as pd

In [35]:
hndata = pd.read_csv("data/hn-results-20170503-113146.csv", encoding="utf8")
hndata.head()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,15,sama,sama,1160423461,1.160423e+09,&#34;the rising star of venture capital&#34; -...,1,NaN,NaN,1
1,82729,vice,vice,1195735854,1.195736e+09,I'm nX 1 too<p>;)\n,1,NaN,True,4
2,487171,jacquesm,jacquesm,1235046083,1.235046e+09,"So, just to see how hard it is to make the lon...",1,NaN,NaN,0
3,234509,kleevr,kleevr,1215030588,1.215031e+09,sequential numbering\n I must be bored,1,NaN,NaN,2
4,454410,Arrington,Arrington,1233174694,1.233175e+09,This is why we can't have nice things.,1,NaN,True,3


In [37]:
hndata.filter(like="comment", axis=0)

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
